# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédiction de **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

// TODO: à enlever à la fin:

Pour importer librairies:
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"], ...)

In [1986]:
# import Pkg; Pkg.add("Plots")
# import Pkg; Pkg.add("DecisionTree")

In [1987]:
# using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, DecisionTree,Random, Base

In [1988]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
        if eltype(stddata[!, col]) <: Number && col != "id"
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

standardize_data (generic function with 1 method)

In [1989]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :general_type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

encode_data (generic function with 1 method)

In [1990]:
function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite, :nombre_cylindres, :annee]))
end

removeRows (generic function with 1 method)

In [1991]:
function add_rows(data)
    # data[!,:volume_gaz] = data[!,:nombre_cylindres] .* data[!,:cylindree]

    # https://www.insurancenavy.com/average-car-weight/
    # https://www.auto-tests.com/fr/lightest-weight/Wagon/all/
    # weight_dict = Dict("voiture_moyenne" => 3300, "VUS_petit" => 3500, "voiture_compacte" => 2800, "voiture_deux_places" => 2800, "voiture_minicompacte" => 1500, "VUS_standard" => 5000, "monospace" => 4500, "voiture_sous_compacte" => 2600, "camionnette_petit" => 4200, "break_petit" => 2640, "voiture_grande" => 4400, "camionnette_standard" => 4700, "break_moyen" => 3300)
    # data[!, :weight] = [weight_dict[t] for t in data[!, :type]]

    # general_type_dict = Dict("voiture_moyenne" => "voiture", "VUS_petit" => "VUS", "voiture_compacte" => "voiture", "voiture_deux_places" => "voiture", "voiture_minicompacte" => "voiture", "VUS_standard" => "VUS", "monospace" => "camionnette", "voiture_sous_compacte" => "voiture", "camionnette_petit" => "camionnette", "break_petit" => "break", "voiture_grande" => "voiture", "camionnette_standard" => "camionnette", "break_moyen" => "break")
    # data[!, :general_type] = [general_type_dict[t] for t in data[!, :type]]
    
    # display(plot(data, x=:general_type, y=:consommation, color=:type))
    # println(combine(groupby(data, :type), nrow => :count))
    return data
end

add_rows (generic function with 1 method)

In [1992]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = add_rows(data_copy)
    standardised_data = standardize_data(data_copy)
    # standardised_data = encode_data(standardised_data)
    standardised_data = removeRows(standardised_data)
    
    print(standardised_data)
    return standardised_data
end




getStandardEncodedData (generic function with 1 method)

In [1993]:
function rmse(y, ychap)
    return sqrt(mean((ychap .- y).^2))
end


rmse (generic function with 1 method)

In [1994]:
function rsquared(y, ychap)
    ss_total = sum((y .- mean(y)).^2)
    ss_res = sum((y .- ychap).^2)
    return ss_res / ss_total
end


rsquared (generic function with 1 method)

In [1995]:
function train_test_split(data, test_size=0.2, shuffle=true)
    n = size(data, 1)
    test_size = floor(Int, n * test_size)
    
    if shuffle
        indices = randperm(n)
    else
        indices = 1:n
    end
    
    test_indices = indices[1:test_size]
    train_indices = indices[test_size+1:end]
    
    train_data = data[train_indices, :]
    test_data = data[test_indices, :]
    
    return train_data, test_data
end

train_test_split (generic function with 3 methods)

In [1996]:
trainData = CSV.read("./data/train.csv", DataFrame)
testData = CSV.read("./data/test.csv", DataFrame)
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))


COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)
trainData[!, :id] = 1:nrow(trainData)
testData[!, :id] = 1:nrow(testData)

possibles = unique(trainData.consommation)


25-element Vector{Float64}:
 13.8358823529412
  9.80041666666667
 11.7605
 13.0672222222222
  7.3503125
  7.58741935483871
 11.2004761904762
 14.700625
 12.3794736842105
 10.6913636363636
  9.4084
 15.6806666666667
  8.71148148148148
  8.11068965517241
 10.2265217391304
  7.84033333333333
  8.40035714285714
  9.04653846153846
 16.8007142857143
  6.91794117647059
  6.35702702702703
  4.52326923076923
  7.12757575757576
  4.90020833333333
  6.53361111111111

In [1997]:
function evaluate_rmse(data)
    nrange = 1000
    n = 0
    for i in range(0, 1, length=nrange)
        n += regression(data)[1]
    end
    print("average rmse for regression: ", n/nrange, "\n")

    # n = 0
    # for i in range(0, 1, length=nrange)
    #     n += ridge_regression(data, 0.1, 0.05)[1]
    # end
    # print("average rmse for ridge: ", n/nrange, "\n")

    # n = 0
    # for i in range(0, 1, length=nrange)
    #     n += svd_regression(data, 0.05)[1]
    # end
    # print("average rmse for svd: ", n/nrange, "\n")

    # nrange = 1000
    # n = 0
    # for i in range(0, 1, length=nrange)
    #     n += polynomial_regression(data,2, 0.05)[1]
    # end
    # print("average rmse for polynomial regression: ", n/nrange, "\n")

end


evaluate_rmse (generic function with 1 method)

In [1998]:
function remise(prediction)
    id = 1:150
    df_pred = DataFrame(id=id, consommation=prediction)

   format_file(df_pred)
end

function format_file(predictions)
    println(predictions)
    current_time = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
    file_name = "benchmark_" * current_time * ".csv"
    CSV.write("./soumissions_potentielles/" *file_name, predictions)
end

format_file (generic function with 1 method)

In [1999]:
# findnearest(A::Vector{Float64},t::Float64) = A[findmin(broadcast(abs,A.-t))[2]]

# function findnearest(A::Vector{Float64},t::Float64)
#     nearest_value = A[1]
#     smallest_diff = abs(A[1] - t)
#     for value in A
#         diff = abs(value - t)
#         if diff < smallest_diff
#             smallest_diff = diff
#             nearest_value = value
#         end
#     end
#     if smallest_diff > 0.3
#         # println("Warning: the nearest value is ", nearest_value, " the value was ", t," with a difference of ", smallest_diff)
#     end
#     return nearest_value
    
# end


In [2000]:
COLUMNS_TO_COMPARE = ["type", "nombre_cylindres", "cylindree", "transmission", "boite"]

function get_unique_data(data)
    unique_data = combine(groupby(data, COLUMNS_TO_COMPARE), :consommation => mean, :annee => first)
    rename!(unique_data, :consommation_mean => :consommation)
    rename!(unique_data, :annee_first => :annee)
    return unique_data
end

function find_pairs(data_train, data_predict)
    train_data_without_consommation = select(data_train, Not(:consommation, :annee))
    # print(train_data_without_consommation)
    predict_data_without_annee = select(data_predict, Not(:annee))
    # print(predict_data_without_annee)

    common_values = innerjoin(data_train, predict_data_without_annee, on=names(train_data_without_consommation))

    println(common_values.id)
    # print(common_values)
    return common_values
end

function find_differents(data_train, data_predict)
    different_values = antijoin(data_predict, data_train, on=COLUMNS_TO_COMPARE)
    return different_values
    
end

function predict_with_dups(data_train, data_predict, raw_data_train, raw_data_predict)
    pairs = find_pairs(raw_data_train, raw_data_predict)
    pairs = select(pairs, [:id, :consommation])

    # println(first(raw_data_train,1))
    # println(first(raw_data_predict,1))

    # println(pairs)

    different_values = find_differents(raw_data_train, raw_data_predict)
    # println(different_values)

    beta = regression(data_train, 0.0)[2]

    data_predict = getStandardEncodedData(different_values[!, Not(:id)])


    data_predict = Matrix(data_predict[:, names(data_train[:, Not(:consommation)])])

    prediction_differents = data_predict * beta

    # print(prediction)


    prediction_differents = (prediction_differents .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN

    prediction_differents = DataFrame(id=different_values.id, consommation=prediction_differents)

    # print(prediction_differents)
    # print(pairs)

    prediction = vcat(pairs, prediction_differents)
    prediction = sort(prediction, :id)
    # println(prediction)

    # print(prediction)
    return prediction
end
    
    

predict_with_dups (generic function with 2 methods)

In [2001]:
function linear_regression(data, x_col, y_col)
    # Extract x and y data from the DataFrame
    x = data[!, x_col]
    y = data[!, y_col]
    
    # Ensure there is enough data for the calculation
    n = length(x)
    if n == 0
        error("Cannot compute linear regression with zero elements.")
    end
    
    # Calculate the mean values
    x_mean = mean(x)
    y_mean = mean(y)
    
    # Calculate slope and intercept for linear regression
    slope = sum((x .- x_mean) .* (y .- y_mean)) / sum((x .- x_mean).^2)
    intercept = y_mean - slope * x_mean
    
    return slope, intercept
end

linear_regression (generic function with 1 method)

In [2002]:

function get_outliers_ind_regression_lin(data, x_col, y_col; threshold=2.5)
    slope, intercept = linear_regression(data, x_col, y_col)
    
    x = data[!, x_col]
    y = data[!, y_col]
    
    y_pred = slope .* x .+ intercept
    
    residuals = abs.(y .- y_pred)
    
    residuals_std = std(residuals)
    
    outlier_indices = findall(residuals .> threshold * residuals_std)
    

    for idx in outlier_indices
        # println("Outlier detected at row $idx: ", "x = ", x[idx], ", y = ", y[idx], ", Residual = ", residuals[idx])
    end

    outliers = data[outlier_indices, :]
    
    return outlier_indices
end


get_outliers_ind_regression_lin (generic function with 1 method)

In [2003]:
function get_outliers_regression_lin(data, outliers_indices)
    outliers = data[outliers_indices, :]
    return outliers
end;

In [2004]:
function remove_outliers_regression_lin(data::DataFrame, x_col::Symbol, y_col::Symbol; threshold=2.5)
    outlier_indices = get_outliers_ind_regression_lin(data, x_col, y_col, threshold=threshold)

    # println("Outlier Indices Identified: ", outlier_indices)

    keep_mask = trues(nrow(data))
    
    if !isempty(outlier_indices)
        keep_mask[outlier_indices] .= false
    end

    cleaned_data = data[keep_mask, :]
    
    return cleaned_data
end

remove_outliers_regression_lin (generic function with 1 method)

In [2005]:
 function plot_outliers(uniqueD)
 # Step 2: Find and Display Outliers with Regression
 outliers_indices = get_outliers_ind_regression_lin(uniqueD, :cylindree, :consommation)
 outliers_regression = get_outliers_regression_lin(uniqueD, outliers_indices)

 # Step 3: Plot Original Data, Regression Line, and Outliers
 slope, intercept = linear_regression(uniqueD, :cylindree, :consommation)
 regression_line_y = slope .* uniqueD.cylindree .+ intercept

 layer_original = layer(x=uniqueD.cylindree, y=uniqueD.consommation, color=uniqueD.type, Theme(default_color="blue"))
 layer_linear_regression = layer(x=uniqueD.cylindree, y=regression_line_y, Geom.line, Theme(default_color="green"))
 layer_regression_outliers = layer(x=outliers_regression.cylindree, y=outliers_regression.consommation, Geom.point, Theme(default_color="red"))

 
 display(plot(layer_regression_outliers, layer_original, layer_linear_regression, Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Original Data with Outliers")))

 # Step 4: Remove Outliers Based on Regression Line
 cleaned_data = remove_outliers_regression_lin(uniqueD, :cylindree, :consommation, threshold=2.5)

 # Step 5: Plot Cleaned Data with New Regression Line
 slope_cleaned, intercept_cleaned = linear_regression(cleaned_data, :cylindree, :consommation)
 regression_line_y_cleaned = slope_cleaned .* cleaned_data.cylindree .+ intercept_cleaned

layer_cleaned = layer(x=cleaned_data.cylindree, y=cleaned_data.consommation, color=cleaned_data.type, Theme(default_color="blue"))
layer_linear_regression_cleaned = layer(x=cleaned_data.cylindree, y=regression_line_y_cleaned, Geom.line, Theme(default_color="green"))

# Display plot for cleaned data
display(plot(layer_cleaned, layer_linear_regression_cleaned, 
             Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Cleaned Data with Regression Line")))

end

plot_outliers (generic function with 1 method)

In [2006]:
function regression(standardised_data, trainTestSplitPercentage = 0.2)
    training_data, test_data = train_test_split(standardised_data, trainTestSplitPercentage)

    X_train =  Matrix(training_data[:, Not(:consommation)])
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0
    if trainTestSplitPercentage != 0.0
        X_test = Matrix(test_data[:, Not(:consommation)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, ychap)
    end
    
    return rmseval, beta, training_data, test_data
end

regression (generic function with 2 methods)

In [2007]:
function remise_regression(beta)
    test_data = getStandardEncodedData(testData)
    X_test = Matrix(test_data[!,Not(:id)])
    ychap =  X_test * beta
    # ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    remise(ychap)
end


remise_regression (generic function with 1 method)

In [ ]:
function main()
    # println(find_duplicates(trainData))

    # Step 1: Get Unique Data
    uniqueD = get_unique_data(trainData)
    # println(uniqueD)
    uniqueD = remove_outliers_regression_lin(uniqueD, :cylindree, :consommation, threshold=2.5)

    unique_standard = getStandardEncodedData(uniqueD)
    predictStandard = getStandardEncodedData(testData)
    # evaluate_rmse(uniqueD)

    # println(uniqueD)
    # println(predictStandard)


    # format_file(predict_with_dups(unique_standard, predictStandard, uniqueD, testData))


    predict_with_dups(unique_standard, predictStandard, uniqueD, testData)
    # format_file(predict_with_dups(uniqueD, testData))
 
end
main()

135×2 DataFrame
 Row │ cylindree   consommation 
     │ Float64     Float64      
─────┼──────────────────────────
   1 │  1.0181       1.14663
   2 │ -0.799335    -0.170046
   3 │  0.185109     0.448869
   4 │  1.47246      1.17575
   5 │ -1.17797     -1.37654
   6 │ -1.17797     -1.33454
   7 │  0.563742     0.320596
   8 │  0.185109     0.586143
   9 │  1.47246      1.92401
  10 │ -0.0420703    0.190856
  11 │ -0.0420703    0.00993906
  12 │  0.412289     0.320596
  13 │ -0.799335    -0.108535
  14 │  1.47246      1.17575
  15 │ -0.0420703    0.320596
  16 │ -0.420703    -0.839304
  17 │  0.563742     0.860692
  18 │  2.154        2.37296
  19 │ -0.799335    -0.781234
  20 │  2.38118      1.92401
  21 │ -0.0420703   -0.220211
  22 │ -1.10224     -1.09483
  23 │  0.336562    -0.12557
  24 │ -0.496429    -0.0482515
  25 │ -0.799335    -0.549646
  26 │ -0.799335    -0.946208
  27 │ -0.420703    -0.443894
  28 │ -0.799335    -0.742862
  29 │ -0.799335    -1.27661
  30 │  2.22972      2.

"./soumissions_potentielles/benchmark_2024-12-01_15-46-23.csv"

In [2009]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
#     println()
# end

In [2010]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     display(plot(x=data_type.volume_gaz, y=data_type.consommation))
#     println()
# end

## Exploration des données